<a href="https://colab.research.google.com/github/castillosebastian/Taller_R_EstadisticaJudicial/blob/master/NLU_Semantica_Vectorial_en_Textos_Legales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantica Vectorial en Textos Legales: Exploración

In [1]:
# Libraries, tools and thanks 
import bs4
import nltk
import numpy
import pandas as pd
import io
import requests
from google.colab import drive
import numpy as np 
import json

In [2]:
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [3]:
%cd  'drive/My Drive/Colab Notebooks/data'

/content/drive/My Drive/Colab Notebooks/data


In [4]:
%ls

corpus_fallosmetdat.json  tbdoctrina.json  tbmetdat.json
corpus_textosfallos.json  tbfallos.json


# Lectura de tablas de datos primarios

In [5]:
with open('tbfallos.json', 'r') as myfile:
    data1=myfile.read()
# parse file
tbfallos = json.loads(data1)
tbfallos = pd.DataFrame.from_dict(tbfallos)

In [6]:
tbfallos['textos_fallo'][1]

'#TEXTO COMPLETO#A C U E R D O#En la ciudad de La Plata, a 15 de mayo de 2019, habiéndose establecido, de conformidad con lo dispuesto en el Acuerdo 2078, que deberá observarse el siguiente orden de votación: doctores Soria, de Lázzari, Genoud, Kogan, se reúnen los señores Jueces de la Suprema Corte de Justicia en acuerdo ordinario para pronunciar sentencia definitiva en la causa C. 122.321, "F., R.. Determinación de la capacidad jurídica".#A N T E C E D E N T E S#La Sala I de la Cámara de Apelación en lo Civil y Comercial de San Isidro confirmó el fallo de primera instancia que, en los autos sobre determinación de la capacidad jurídica del señor R. F., resolvió aprobar la rendición de cuentas efectuada por la Curaduría Oficial de Alienados (v. fs. 689 y 660, respectivamente).#Se interpuso, por la titular de la Asesoría de Incapaces n° 2 departamental, recurso extraordinario de inaplicabilidad de ley (v. fs. 705/711).#Oído el señor Procurador General, dictada la providencia de autos y 

In [7]:
with open('tbmetdat.json', 'r') as myfile:
    data1=myfile.read()
# parse file
tbmetdat = json.loads(data1)
tbmetdat = pd.DataFrame.from_dict(tbmetdat)

In [8]:
tbmetdat[tbmetdat.nro_causa == 122321]

,nro_causa,fres,textos_raw_len,materia,tipo_fallo,tribunal_emisor,caratula,caratula_pub,magistrados_votantes,tribunal_origen,bnro,magistrados,sumario,voces
1,122321,2019-05-15,160,CIVIL Y COMERCIAL,Sentencia Definitiva,SUPREMA CORTE DE JUSTICIA DE LA PROVINCIA (SCBA),"Figueredo, Rafael s/ Determinación de la capac...","F. ,R. s/ Determinación de la capacidad jurídica",Soria-de Lázzari-Genoud-Kogan,CAMARA DE APELACION EN LO CIVIL Y COMERCIAL SA...,158.0,Soria - de Lázzari - Genoud - Kogan.,\nSentencia-Fundamentación. Rendición de cuen...,"RENDICION DE CUENTAS,APROBACION,SENTENCIA,FUND..."


In [9]:
tbmetdat.loc[1]

nro_causa                                                          122321
fres                                                           2019-05-15
textos_raw_len                                                        160
materia                                                 CIVIL Y COMERCIAL
tipo_fallo                                           Sentencia Definitiva
tribunal_emisor          SUPREMA CORTE DE JUSTICIA DE LA PROVINCIA (SCBA)
caratula                Figueredo, Rafael s/ Determinación de la capac...
caratula_pub             F. ,R. s/ Determinación de la capacidad jurídica
magistrados_votantes                        Soria-de Lázzari-Genoud-Kogan
tribunal_origen         CAMARA DE APELACION EN LO CIVIL Y COMERCIAL SA...
bnro                                                                  158
magistrados                          Soria - de Lázzari - Genoud - Kogan.
sumario                  \nSentencia-Fundamentación. Rendición de cuen...
voces                   RENDICION DE C

In [10]:
tbmetdat.loc[1].sumario

' \nSentencia-Fundamentación. Rendición de cuentas-Aprobación. \nLa Suprema Corte de Justicia de la Provincia de Buenos Aires revocó la sentencia que tubo por \naprobada  la rendición  de  cuentas  presentada  por  el  Curador  Oficial  en  el  marco  del  proceso de \ndeterminación de la capacidad, considerando la falta de fundamentación suficiente del fallo y de \nadecuación a los hechos probados en la causa. '

In [11]:
with open('tbdoctrina.json', 'r') as myfile:
    data1=myfile.read()
# parse file
tbdoctrina = json.loads(data1)
tbdoctrina = pd.DataFrame.from_dict(tbdoctrina)

In [12]:
tbdoctrina.head()

,nro_causa,doctrina_nro,doctrina
0,122514,1,El recurso de inaplicabilidad de ley no consti...
1,122514,2,Los juicios de divorcio en trámite al momento ...
2,122514,3,Cuando en la causa sobre divorcio las partes n...
3,122514,4,El artículo 7 del nuevo Código Civil y Comerci...
4,122514,5,La aplicación inmediata de las normas se disti...


In [13]:
tbdoctrina[tbdoctrina.nro_causa == 122321].doctrina

10    Debe ser revocado el fallo que, como único sus...
Name: doctrina, dtype: object

# Procesamiento con Stanza (StanfordNLP)

- https://github.com/stanfordnlp/stanza

In [ ]:
#! pip install stanza

In [15]:
import stanza
stanza.download('es') # download English model
nlp = stanza.Pipeline('es') # initialize Spanish neural pipeline
doc = nlp("Alberto Fernandez es presidente de Argetina") # run annotation over a sentence

2021-10-18 12:32:47 INFO: Downloading default packages for language: es (Spanish)...


2021-10-18 12:33:03 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-10-18 12:33:03 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2021-10-18 12:33:03 INFO: Use device: cpu
2021-10-18 12:33:03 INFO: Loading: tokenize
2021-10-18 12:33:03 INFO: Loading: mwt
2021-10-18 12:33:03 INFO: Loading: pos
2021-10-18 12:33:04 INFO: Loading: lemma
2021-10-18 12:33:04 INFO: Loading: depparse
2021-10-18 12:33:04 INFO: Loading: ner
2021-10-18 12:33:05 INFO: Done loading processors!


In [16]:
doc

[
  [
    {
      "id": 1,
      "text": "Alberto",
      "lemma": "Alberto",
      "upos": "PROPN",
      "head": 4,
      "deprel": "nsubj",
      "start_char": 0,
      "end_char": 7,
      "ner": "B-PER"
    },
    {
      "id": 2,
      "text": "Fernandez",
      "lemma": "Fernandez",
      "upos": "PROPN",
      "head": 1,
      "deprel": "flat",
      "start_char": 8,
      "end_char": 17,
      "ner": "E-PER"
    },
    {
      "id": 3,
      "text": "es",
      "lemma": "ser",
      "upos": "AUX",
      "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin",
      "head": 4,
      "deprel": "cop",
      "start_char": 18,
      "end_char": 20,
      "ner": "O"
    },
    {
      "id": 4,
      "text": "presidente",
      "lemma": "presidente",
      "upos": "NOUN",
      "feats": "Gender=Masc|Number=Sing",
      "head": 0,
      "deprel": "root",
      "start_char": 21,
      "end_char": 31,
      "ner": "O"
    },
    {
      "id": 5,
      "text": "de",
      "lemma

# Tokenizacion y Segmentacion de sentencias

## Método por REGEX

In [17]:
import re
def split_into_sentences(text):       
    text = text.replace("#","<stop>")    
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences
split_into_sentences(tbfallos['textos_fallo'][1])

['',
 'TEXTO COMPLETO',
 'A C U E R D O',
 'En la ciudad de La Plata, a 15 de mayo de 2019, habiéndose establecido, de conformidad con lo dispuesto en el Acuerdo 2078, que deberá observarse el siguiente orden de votación: doctores Soria, de Lázzari, Genoud, Kogan, se reúnen los señores Jueces de la Suprema Corte de Justicia en acuerdo ordinario para pronunciar sentencia definitiva en la causa C. 122.321, "F., R.. Determinación de la capacidad jurídica".',
 'A N T E C E D E N T E S',
 'La Sala I de la Cámara de Apelación en lo Civil y Comercial de San Isidro confirmó el fallo de primera instancia que, en los autos sobre determinación de la capacidad jurídica del señor R. F., resolvió aprobar la rendición de cuentas efectuada por la Curaduría Oficial de Alienados (v. fs. 689 y 660, respectivamente).',
 'Se interpuso, por la titular de la Asesoría de Incapaces n° 2 departamental, recurso extraordinario de inaplicabilidad de ley (v. fs. 705/711).',
 'Oído el señor Procurador General, dicta

## Método Stanza

In [18]:
text = tbfallos['textos_fallo'][1]
text = text.replace("#","\n\n")

In [19]:
nlp = stanza.Pipeline(lang='es', processors='tokenize', tokenize_no_ssplit=True)
doc = nlp(text)
for i, sentence in enumerate(doc.sentences):
    print(f'====== Sentence {i+1} tokens =======')
    print(*[f'id: {token.id}\ttext: {token.text}' for token in sentence.tokens], sep='\n')

2021-10-18 12:33:25 WARNING: Language es package default expects mwt, which has been added
2021-10-18 12:33:25 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |

2021-10-18 12:33:25 INFO: Use device: cpu
2021-10-18 12:33:25 INFO: Loading: tokenize
2021-10-18 12:33:25 INFO: Loading: mwt
2021-10-18 12:33:25 INFO: Done loading processors!
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


====== Sentence 1 tokens =======
id: (1,)	text: TEXTO
id: (2,)	text: COMPLETO
====== Sentence 2 tokens =======
id: (1,)	text: A
id: (2,)	text: C
id: (3,)	text: U
id: (4,)	text: E
id: (5,)	text: R
id: (6,)	text: D
id: (7,)	text: O
====== Sentence 3 tokens =======
id: (1,)	text: En
id: (2,)	text: la
id: (3,)	text: ciudad
id: (4,)	text: de
id: (5,)	text: La
id: (6,)	text: Plata
id: (7,)	text: ,
id: (8,)	text: a
id: (9,)	text: 15
id: (10,)	text: de
id: (11,)	text: mayo
id: (12,)	text: de
id: (13,)	text: 2019
id: (14,)	text: ,
id: (15, 16)	text: habiéndose
id: (17,)	text: establecido
id: (18,)	text: ,
id: (19,)	text: de
id: (20,)	text: conformidad
id: (21,)	text: con
id: (22,)	text: lo
id: (23,)	text: dispuesto
id: (24,)	text: en
id: (25,)	text: el
id: (26,)	text: Acuerdo
id: (27,)	text: 2078
id: (28,)	text: ,
id: (29,)	text: que
id: (30,)	text: deberá
id: (31, 32)	text: observarse
id: (33,)	text: el
id: (34,)	text: siguiente
id: (35,)	text: orden
id: (36,)	text: de
id: (37,)	text: votación

In [ ]:
doc.sentences[11]

# POS y Ufeat

In [ ]:
# nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos')
# print(*[f'word: {word.text}\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}' for sent in doc.sentences for word in sent.words], sep='\n')

# Entidades

In [28]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,ner')
doc = nlp(text)


2021-10-18 12:37:41 WARNING: Language es package default expects mwt, which has been added
2021-10-18 12:37:41 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| ner       | conll02 |

2021-10-18 12:37:41 INFO: Use device: cpu
2021-10-18 12:37:41 INFO: Loading: tokenize
2021-10-18 12:37:41 INFO: Loading: mwt
2021-10-18 12:37:41 INFO: Loading: ner
2021-10-18 12:37:42 INFO: Done loading processors!


In [29]:
print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')


entity: TEXTO COMPLETO	type: MISC
entity: La Plata	type: LOC
entity: Acuerdo 2078	type: MISC
entity: Soria	type: PER
entity: Lázzari	type: PER
entity: Genoud	type: PER
entity: Kogan	type: PER
entity: Suprema Corte de Justicia	type: ORG
entity: C. 122.321	type: MISC
entity: Sala I de la Cámara de Apelación	type: ORG
entity: Civil	type: MISC
entity: Comercial de San Isidro	type: ORG
entity: señor R. F.	type: PER
entity: Curaduría Oficial de Alienados	type: ORG
entity: Asesoría de Incapaces n° 2	type: ORG
entity: Procurador General	type: PER
entity: Suprema Corte	type: ORG
entity: Juez doctor Soria	type: PER
entity: señor R. F.	type: PER
entity: Juzgado de Familia n° 4	type: ORG
entity: San Isidro	type: LOC
entity: Curaduría Oficial de Alienados de San Isidro	type: ORG
entity: Zárate-Campana	type: ORG
entity: Cuenta de Autos	type: MISC
entity: Cuenta Fiscal de Curaduría	type: ORG
entity: señor F.	type: PER
entity: Asesora de Incapaces	type: ORG
entity: Daniela Elizabeth Ferreyra-	type: PE

# Imprimo Palabra

In [23]:
word = doc.sentences[0].words[0]
print(word)

{
  "id": 1,
  "text": "TEXTO",
  "start_char": 2,
  "end_char": 7
}
